In [13]:
import json
from src.resources.integration.fparser_python import *
from src.resources.integration.fparser import *
from src.flasking.sending_windows import *

In [14]:
from pathlib import Path
with open("./CRAM_Plan_Executive.txt", 'r') as f:
    context = f.read()

In [15]:
summary = parse_summary_data(context)

In [18]:
summary[0].items()

dict_items([('Segment 1', 'the person begins preparing for a task by gathering a box of Cheez-It crackers, a red bowl, a red cup, and a bottle of mustard, likely to create a snack or dish.'), ('Segment 2', 'the person approaches the box of Cheez-It crackers to open it, signaling the start of the pouring task.'), ('Segment 3', 'the person grasps the box of Cheez-It crackers with both hands, holding it steady on the table as part of the grasping phase.'), ('Segment 4', 'the person begins grasping-and-moving the box of Cheez-It crackers, pouring them into the red bowl while the red cup and mustard are ready for use.'), ('Segment 5', 'the person continues pouring Cheez-It crackers into the red bowl, maintaining the grasp-and-move phase as they prepare the snack.'), ('Segment 6', 'the person refines the pouring motion, ensuring the crackers are evenly distributed into the bowl while the other items remain in place.'), ('Segment 7', 'the person adjusts the pouring technique, carefully transf

In [4]:
segments = parse_segment_data(context)

In [10]:
segments_str = []
for k,v in  summary[0].items():
    segments_str.append(v)

In [ ]:
segments[0]

In [12]:
type(segments_str)

list

In [ ]:
filtered_segments = filter_redundant_actions(segments)

In [ ]:
len(filtered_segments)

In [ ]:
# final_unique_segments = filter_redundant_executed_actions(filtered_segments)

In [ ]:
chain = decomposer_prompt | ollama_llm.with_structured_output(AtomicsModel, method="json_schema")

In [ ]:
final_parse = decompose_segments_with_atomic_actions(chain, filtered_segments)

In [ ]:
len(final_parse)

In [5]:
chain2 = decomposer_prompt | ollama_llm.with_structured_output(AtomicsModel, method="json_schema")

In [6]:
final_parse = chain2.invoke({'segments': summary[0]})

In [7]:
final_parse = final_parse.atomics

In [ ]:
# final_parse[12]['instruction']
for par in final_parse:
    print(par['instruction'])

In [8]:
from src.langchain.agents.enhanced_ad_agent import *

Trying OLLAMA model 14b
OLLAMA model found


In [ ]:
instruction = final_parse[0]['instruction']
ad = final_parse[0]['action_designator']

In [ ]:
ad

In [9]:
graph_output = []

In [10]:
final_parse_filtered = final_parse

In [11]:
# final_parse[4]

In [12]:
final_parse_filtered

['Pick up the box of Cheez-It crackers',
 'Pick up the red bowl',
 'Pick up the red cup',
 'Pick up the bottle of mustard',
 'Place the red bowl on the table in a stable position',
 'Place the red cup near the red bowl',
 'Place the bottle of mustard near the red bowl',
 'Open the box of Cheez-It crackers',
 'Pour the Cheez-It crackers from the box into the red bowl',
 'Evaluate the pouring process to ensure even distribution',
 'Place the empty box back on the table']

In [13]:
for fp in final_parse_filtered:
    # instruction = fp['instruction']
    # ad = fp['action_designator']
    out = ad_graph.invoke({'instruction': fp, 'context': ""})
    graph_output.append(out)

INSIDE ACTION NODE
type={'action': 'PickingUp'}
['PickingUp']
INSIDE UNIVERSAL NODE
cram_plan (perform (an action (type grab-object) (an object (type {obj_to_be_grabbed}){obj_to_be_grabbed_props}) (source (a location (on {location}){location_props}))))
----------
{"obj_to_be_grabbed": "the box of Cheez-It crackers", "action_verb": "Pick up", "location": null}
----------
fCONTEXT 
{
  "action_verb": "Pick up",
  "obj_to_be_grabbed": "box of Cheez-It crackers",
  "obj_to_be_grabbed_props": {
    "material": "cardboard",
    "texture": "smooth",
    "shape": "rectangular",
    "size": "medium",
    "pattern": "printed",
    "cleanliness": "clean"
  },
  "location": null
}
----------
(perform (an action (type grab-object) (an object (type box of Cheez-It crackers) (material cardboard) (texture smooth) (shape rectangular) (size medium) (pattern printed) (cleanliness clean))))
----------
INSIDE ACTION NODE
type={'action': 'PickingUp'}
['PickingUp']
INSIDE UNIVERSAL NODE
cram_plan (perform (a

In [14]:
for seg in segments:
    print("📡 Sending test data to 1st window...")
    send_data_to_window(1, seg)

📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Window 1: {'status': 'success', 'window': 'window1'}
📡 Sending test data to 1st window...
✅ Data sent to Win

In [15]:
for par in final_parse:
    print("📡 Sending test data to 2nd window...")
    send_data_to_window(2, par)

📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Window 2: {'status': 'success', 'window': 'window2'}
📡 Sending test data to 2nd window...
✅ Data sent to Win

In [ ]:
# graph_output

In [16]:
for out in graph_output:
    print("<UNK> Sending test data to 3rd window...")
    new_out = {}
    try:
        new_out['instruction'] = out['instruction']
        new_out['action_core'] = out['action_core']
        new_out['action_core_attributes'] = json.loads(out['action_core_attributes'])
        new_out['enriched_action_core_attributes'] = json.loads(out['enriched_action_core_attributes'])
        new_out['cram_plan'] = out['cram_plan_response']
        print("Parsed output normally")
    except:
        new_out['instruction'] = out['instruction']
        new_out['action_core'] = out['action_core']
        new_out['action_core_attributes'] = out['action_core_attributes']
        new_out['enriched_action_core_attributes'] = out['enriched_action_core_attributes']
        new_out['cram_plan'] = out['cram_plan_response']
        print("Parsed output with strings")

    send_data_to_window(3, new_out)

<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent to Window 3: {'status': 'success', 'window': 'window3'}
<UNK> Sending test data to 3rd window...
Parsed output normally
✅ Data sent 

In [ ]:
from src.langchain.models_graph import *
config = {"configurable" : {"thread_id" : 1}}

In [ ]:
framenet_model = ""
flanagan = ""

In [ ]:
len(graph_output)

In [ ]:
for out in graph_output:
    print("<UNK> Sending test data to 4th window...")
    instruction = out['instruction']
    action_core = out['action_core']
    action_core_attributes = out['action_core_attributes']
    enriched_action_core_attributes = json.loads(out['enriched_action_core_attributes'])
    cram_plan = out['cram_plan_response']
    final_graph_state = model_graph.invoke({"instruction": instruction,
                                                 "action_core": action_core,
                                                 "enriched_action_core_attributes": enriched_action_core_attributes,
                                                 "cram_plan_response": cram_plan},config=config, stream_mode="updates")
    flanagan = model_graph.get_state(config).values["flanagan"]
    framenet_model = model_graph.get_state(config).values["framenet_model"]
    new_out = {}
    try:
        flanagan_json = json.loads(flanagan)
        framenet_model_json = json.loads(framenet_model)
        print("Parsed models output normally")
        new_out = {
            "framenet" : framenet_model_json,
            "flanagan" : flanagan_json
        }
    except:
        print("Parsed models output with strings")
        new_out = {
            "framenet" : framenet_model,
            "flanagan" : flanagan
        }

    send_data_to_window(4, new_out)

In [ ]:
print(flanagan)

In [ ]:
print(framenet_model)